# One hot save and clean

In [117]:
import pandas as pd
import csv
import os
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from start import data_path
import extract_laws
import clean_documents
import numpy as np

In [146]:
doi_df = pd.read_csv(os.path.join(data_path, 'doi_exemptions_temp.csv'))
for law in doi_df[doi_df.title == "Joaquin ISD"].possible_laws:
    print(law)

[25.112, 25.112, 25.0811a, 25.112a, 21.003a, 25.114, 21.401, 21.057a, 25.113a, 25.113, 25.112]


In [119]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp('(Ed. Code 25.0811)(Ed. Code 25.0812)')
doc[4].shape_

'dd.dddd)(Xx'

In [120]:
nlp = spacy.load('en_core_web_sm')
law_shapes = [i * 'd' + '.' + j * 'd' + k * 'x' for i in range(1, 4) for j in range(3,5) for k in range(3)]
#law_shapes.extend([shape + i * ')' +  m * '(' + n * 'X' + j * 'x' + k * 'd' for shape in law_shapes for i in range(2) for m in range(2) for n in range(2) for j in range(2) for k in range(2)])
law_shapes.extend([shape + i * '(' + j * 'x' + k * 'd' for shape in law_shapes for i in range(2) for j in range(2) for k in range(2)])
law_shapes.append('XXXdd.dddd')
law_shapes.append('xxxdd.dddd')
law_shapes.append('XXXdd.ddd')
law_shapes.append('xxxdd.ddd')
law_shapes

['d.ddd',
 'd.dddx',
 'd.dddxx',
 'd.dddd',
 'd.ddddx',
 'd.ddddxx',
 'dd.ddd',
 'dd.dddx',
 'dd.dddxx',
 'dd.dddd',
 'dd.ddddx',
 'dd.ddddxx',
 'ddd.ddd',
 'ddd.dddx',
 'ddd.dddxx',
 'ddd.dddd',
 'ddd.ddddx',
 'ddd.ddddxx',
 'd.ddd',
 'd.dddd',
 'd.dddx',
 'd.dddxd',
 'd.ddd(',
 'd.ddd(d',
 'd.ddd(x',
 'd.ddd(xd',
 'd.dddx',
 'd.dddxd',
 'd.dddxx',
 'd.dddxxd',
 'd.dddx(',
 'd.dddx(d',
 'd.dddx(x',
 'd.dddx(xd',
 'd.dddxx',
 'd.dddxxd',
 'd.dddxxx',
 'd.dddxxxd',
 'd.dddxx(',
 'd.dddxx(d',
 'd.dddxx(x',
 'd.dddxx(xd',
 'd.dddd',
 'd.ddddd',
 'd.ddddx',
 'd.ddddxd',
 'd.dddd(',
 'd.dddd(d',
 'd.dddd(x',
 'd.dddd(xd',
 'd.ddddx',
 'd.ddddxd',
 'd.ddddxx',
 'd.ddddxxd',
 'd.ddddx(',
 'd.ddddx(d',
 'd.ddddx(x',
 'd.ddddx(xd',
 'd.ddddxx',
 'd.ddddxxd',
 'd.ddddxxx',
 'd.ddddxxxd',
 'd.ddddxx(',
 'd.ddddxx(d',
 'd.ddddxx(x',
 'd.ddddxx(xd',
 'dd.ddd',
 'dd.dddd',
 'dd.dddx',
 'dd.dddxd',
 'dd.ddd(',
 'dd.ddd(d',
 'dd.ddd(x',
 'dd.ddd(xd',
 'dd.dddx',
 'dd.dddxd',
 'dd.dddxx',
 'dd.dddxxd',

In [144]:
doi_df[doi_df.title == "Henrietta ISD"].text.values[0]

'District of Innovation Resolution Board of Education: Principals: Paul Bennett Michael Smiley President High School 720-7930 Billy Johnson Fax 538-7535 Vice President HENRIETTA INDEPENDENT SCHOOL DISTRICT Randy Zamzow "Effective Education Today for a Brighter Tomorrow" Middle School Lanny Evans 720-7920 Jeff McClure, Superintendent Secretary Fax 538-7525 Scot Clayton, Assistant Superintendent Kendra Bennett 1801 E. CRAFTON Nala Chambers ElementaryHENRIETTA, TEXAS 76365 Assistant Secretary 720-7910(940) 720-7900 Fax 538-7515 FAX (940) 538-7505 Resolution To Adopt A District Of Innovation Whereas, Education Code 12A.001 provides that a district is eligible for designation as a district of innovation if the district\'s most recent performance rating under Section 39.054 reflects at least acceptable performance, and that consideration of designation as a district of innovation may be initiated by a resolution adopted by the board of trustees of the district; and Whereas, the Henrietta Ind

In [122]:
doi_df.head()

,title,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,level,type,link,text,p_innovation,possible_laws
0,Abbott ISD,823.0,1941,1941,1941,2139,Second,pdf,https://www.abbottisd.org/ourpages/auto/2018/3...,1 Abbott Independent School District District ...,0.999955,"[25.113, 37.0012, 25.0811, 45.206, 21.102, 21...."
1,Abernathy ISD,822.0,820,820,820,878,Second,pdf,https://1.cdn.edl.io/epGD4mZXjWUcPl8yA7IhlcDol...,Abernathy ISD District of Innovation Plan – Dr...,0.999904,"[37.0012, 28.216, 21.057, 25.112, 25.113, 21.0..."
2,Abilene ISD,821.0,0,0,0,0,First,pdf,https://www.abileneisd.org/wp-content/uploads/...,0 ABILENE INDEPENDENT SCHOOL DISTRICT Local In...,0.999955,"[25.0811, 21.053, 21.003]"
3,Academy ISD,820.0,893,893,893,962,Second,docx,https://4.files.edl.io/1a8f/06/29/18/204245-44...,District of Innovation Plan Developed in coope...,0.999208,"[37.0012, 21.003, 21.053, 21.051, 21.102, 25.0..."
4,Adrian ISD,819.0,125,125,125,125,First,pdf,http://www.adrianisd.net/UserFiles/Servers/Ser...,Adrian ISD District of Innovation Plan House B...,0.999955,"[25.092, 28.216, 25.081, 21.102, 21.003, 21.40..."


## Save create one-hot variables for laws

In [123]:
def make_list(string):
    new_list = [i.replace('[','').replace(']','') for i in string.split(', ')]
    return new_list

doi_df['possible_laws'] = doi_df['possible_laws'].apply(make_list)
doi_df.head(11)

,title,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,level,type,link,text,p_innovation,possible_laws
0,Abbott ISD,823.0,1941,1941,1941,2139,Second,pdf,https://www.abbottisd.org/ourpages/auto/2018/3...,1 Abbott Independent School District District ...,0.999955,"[25.113, 37.0012, 25.0811, 45.206, 21.102, 21...."
1,Abernathy ISD,822.0,820,820,820,878,Second,pdf,https://1.cdn.edl.io/epGD4mZXjWUcPl8yA7IhlcDol...,Abernathy ISD District of Innovation Plan – Dr...,0.999904,"[37.0012, 28.216, 21.057, 25.112, 25.113, 21.0..."
2,Abilene ISD,821.0,0,0,0,0,First,pdf,https://www.abileneisd.org/wp-content/uploads/...,0 ABILENE INDEPENDENT SCHOOL DISTRICT Local In...,0.999955,"[25.0811, 21.053, 21.003]"
3,Academy ISD,820.0,893,893,893,962,Second,docx,https://4.files.edl.io/1a8f/06/29/18/204245-44...,District of Innovation Plan Developed in coope...,0.999208,"[37.0012, 21.003, 21.053, 21.051, 21.102, 25.0..."
4,Adrian ISD,819.0,125,125,125,125,First,pdf,http://www.adrianisd.net/UserFiles/Servers/Ser...,Adrian ISD District of Innovation Plan House B...,0.999955,"[25.092, 28.216, 25.081, 21.102, 21.003, 21.40..."
5,Agua Dulce ISD,818.0,627,627,627,658,Second,pdf,https://tx02206063.schoolwires.net/cms/lib/TX0...,Microsoft Word - DOI 2017-2022.docx Agua Dulce...,0.999955,"[21.053, 21.003, 21.401, 21.057, 25.036, 25.08..."
6,Alamo Heights ISD,817.0,2586,2586,2586,2876,Second,pdf,http://www.ahisd.net/UserFiles/Servers/Server_...,1 | P a g e Mission Statement The Alamo Height...,0.999955,"[37.0012, 25.0811, 25.111, 21.044, 21.102, 25...."
7,Albany ISD,816.0,438,438,438,438,Second,pdf,http://www.albanyisd.net/uploads/4/4/4/1/44419...,Microsoft Word - innovation_plan.docx Albany I...,0.999955,"[25.112, 21.057, 21.003, 25.0811, 25.113]"
8,Aldine ISD,815.0,1974,1974,1974,2176,Second,pdf,http://insidealdine.com/wp-content/uploads/201...,Aldine District of Innovation Plan Approved by...,0.999955,"[25.0811, 11.1511, 25.084, 21.003, 21.353, 21...."
9,Aledo ISD,130.0,1682,1682,1682,1854,Second,pdf,https://docs.google.com/viewerng/viewer?url=ht...,ALEDO Independent School District A Past to Re...,0.329979,"[21.003, 37.0012, 21.3541, 21.053, 21.352, 25...."


In [124]:
mlb = MultiLabelBinarizer()
plans_onehot = doi_df.join(pd.DataFrame(mlb.fit_transform(doi_df.pop('possible_laws')),
                          columns=mlb.classes_,
                          index=doi_df.index))
plans_onehot.head(10)

,title,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,level,type,link,text,...,7.056,7.057,75.600,921.406,97.1001,TEC11.251,TEC21.003,TEC21.057,TEC21.354,TEC25.112
0,Abbott ISD,823.0,1941,1941,1941,2139,Second,pdf,https://www.abbottisd.org/ourpages/auto/2018/3...,1 Abbott Independent School District District ...,...,0,0,0,0,0,0,0,0,0,0
1,Abernathy ISD,822.0,820,820,820,878,Second,pdf,https://1.cdn.edl.io/epGD4mZXjWUcPl8yA7IhlcDol...,Abernathy ISD District of Innovation Plan – Dr...,...,0,0,0,0,0,0,0,0,0,0
2,Abilene ISD,821.0,0,0,0,0,First,pdf,https://www.abileneisd.org/wp-content/uploads/...,0 ABILENE INDEPENDENT SCHOOL DISTRICT Local In...,...,0,0,0,0,0,0,0,0,0,0
3,Academy ISD,820.0,893,893,893,962,Second,docx,https://4.files.edl.io/1a8f/06/29/18/204245-44...,District of Innovation Plan Developed in coope...,...,0,0,0,0,0,0,0,0,0,0
4,Adrian ISD,819.0,125,125,125,125,First,pdf,http://www.adrianisd.net/UserFiles/Servers/Ser...,Adrian ISD District of Innovation Plan House B...,...,0,0,0,0,0,0,0,0,0,0
5,Agua Dulce ISD,818.0,627,627,627,658,Second,pdf,https://tx02206063.schoolwires.net/cms/lib/TX0...,Microsoft Word - DOI 2017-2022.docx Agua Dulce...,...,0,0,0,0,0,0,0,0,0,0
6,Alamo Heights ISD,817.0,2586,2586,2586,2876,Second,pdf,http://www.ahisd.net/UserFiles/Servers/Server_...,1 | P a g e Mission Statement The Alamo Height...,...,0,0,0,0,0,0,0,0,0,0
7,Albany ISD,816.0,438,438,438,438,Second,pdf,http://www.albanyisd.net/uploads/4/4/4/1/44419...,Microsoft Word - innovation_plan.docx Albany I...,...,0,0,0,0,0,0,0,0,0,0
8,Aldine ISD,815.0,1974,1974,1974,2176,Second,pdf,http://insidealdine.com/wp-content/uploads/201...,Aldine District of Innovation Plan Approved by...,...,0,0,0,0,0,0,0,0,0,0
9,Aledo ISD,130.0,1682,1682,1682,1854,Second,pdf,https://docs.google.com/viewerng/viewer?url=ht...,ALEDO Independent School District A Past to Re...,...,0,0,0,0,0,0,0,0,0,0


In [125]:
drop_cols = [c for c in plans_onehot.columns if c.lower()[:7] == 'unnamed']
plans_onehot = plans_onehot.drop(drop_cols, axis=1)
plans_onehot.head()

,title,level,type,link,text,p_innovation,,1.251,1.253,102.1037(d,...,7.056,7.057,75.600,921.406,97.1001,TEC11.251,TEC21.003,TEC21.057,TEC21.354,TEC25.112
0,Abbott ISD,Second,pdf,https://www.abbottisd.org/ourpages/auto/2018/3...,1 Abbott Independent School District District ...,0.999955,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Abernathy ISD,Second,pdf,https://1.cdn.edl.io/epGD4mZXjWUcPl8yA7IhlcDol...,Abernathy ISD District of Innovation Plan – Dr...,0.999904,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Abilene ISD,First,pdf,https://www.abileneisd.org/wp-content/uploads/...,0 ABILENE INDEPENDENT SCHOOL DISTRICT Local In...,0.999955,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Academy ISD,Second,docx,https://4.files.edl.io/1a8f/06/29/18/204245-44...,District of Innovation Plan Developed in coope...,0.999208,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Adrian ISD,First,pdf,http://www.adrianisd.net/UserFiles/Servers/Ser...,Adrian ISD District of Innovation Plan House B...,0.999955,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Rename exemptions so they start with reg

In [126]:
plans_onehot.columns = ['reg' + str(i).replace('.', '_') for i in plans_onehot.columns]
plans_onehot = plans_onehot.drop(['reg'], axis=1)
plans_onehot = plans_onehot.rename(index=str, columns= {'regtitle':'district', 
                                                        'reglink': 'link',
                                                        'regtext': 'text',
                                                        'regp_innovation':'p_doi'})
plans_onehot.head(10)

,district,reglevel,regtype,link,text,p_doi,reg1_251,reg1_253,reg102_1037(d,reg102_1301,...,reg7_056,reg7_057,reg75_600,reg921_406,reg97_1001,regTEC11_251,regTEC21_003,regTEC21_057,regTEC21_354,regTEC25_112
0,Abbott ISD,Second,pdf,https://www.abbottisd.org/ourpages/auto/2018/3...,1 Abbott Independent School District District ...,0.999955,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Abernathy ISD,Second,pdf,https://1.cdn.edl.io/epGD4mZXjWUcPl8yA7IhlcDol...,Abernathy ISD District of Innovation Plan – Dr...,0.999904,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Abilene ISD,First,pdf,https://www.abileneisd.org/wp-content/uploads/...,0 ABILENE INDEPENDENT SCHOOL DISTRICT Local In...,0.999955,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Academy ISD,Second,docx,https://4.files.edl.io/1a8f/06/29/18/204245-44...,District of Innovation Plan Developed in coope...,0.999208,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Adrian ISD,First,pdf,http://www.adrianisd.net/UserFiles/Servers/Ser...,Adrian ISD District of Innovation Plan House B...,0.999955,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Agua Dulce ISD,Second,pdf,https://tx02206063.schoolwires.net/cms/lib/TX0...,Microsoft Word - DOI 2017-2022.docx Agua Dulce...,0.999955,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Alamo Heights ISD,Second,pdf,http://www.ahisd.net/UserFiles/Servers/Server_...,1 | P a g e Mission Statement The Alamo Height...,0.999955,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Albany ISD,Second,pdf,http://www.albanyisd.net/uploads/4/4/4/1/44419...,Microsoft Word - innovation_plan.docx Albany I...,0.999955,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Aldine ISD,Second,pdf,http://insidealdine.com/wp-content/uploads/201...,Aldine District of Innovation Plan Approved by...,0.999955,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Aledo ISD,Second,pdf,https://docs.google.com/viewerng/viewer?url=ht...,ALEDO Independent School District A Past to Re...,0.329979,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [127]:
cols = [c for c in plans_onehot.columns if c.lower()[:3] == 'reg']
plans_onehot['total'] = plans_onehot[cols].sum(axis = 1)
print(len(plans_onehot))
# drop documents with missing regulations
plans_onehot = plans_onehot[plans_onehot.total > 0]
print(len(plans_onehot))

824
811


In [128]:
# replace values
plans_onehot['reg25_0811'] = np.where((plans_onehot.reg25_081 == 1), 1, plans_onehot.reg25_0811)

# Save

In [129]:
plans_onehot.to_csv((os.path.join(data_path, 'doi_final_wtext.csv')))
plans_onehot = plans_onehot.drop(['text'], axis = 1)
plans_onehot.to_csv((os.path.join(data_path, 'doi_final.csv')))

In [130]:
df = pd.read_csv((os.path.join(data_path, 'doi_final.csv')))

In [131]:
plans_onehot.reg25_0811.mean()

0.967940813810111

In [132]:
df[df.reg25_0811 == 0]

,Unnamed: 0,district,reglevel,regtype,link,p_doi,reg1_251,reg1_253,reg102_1037(d,reg102_1301,...,reg7_057,reg75_600,reg921_406,reg97_1001,regTEC11_251,regTEC21_003,regTEC21_057,regTEC21_354,regTEC25_112,total
171,172,Coolidge ISD,Second,google,https://docs.google.com/document/d/1CgtVWNKgd3...,0.999955,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
207,209,Deweyville ISD,First,pdf,https://deweyvilleisd.com/files/user/2/file/Di...,0.999561,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
227,232,Elgin ISD,Second,pdf,http://sww.elginisd.net/cms/lib6/TX01917830/Ce...,0.991237,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
267,273,Fruitvale ISD,First,pdf,http://www.fruitvaleisd.com/upload/page/0132/d...,0.999046,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
281,287,Gladewater ISD,First,pdf,http://www.gladewaterisd.com/upload/page/0151/...,0.999955,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
333,339,Henrietta ISD,First,pdf,http://www.henrietta-isd.net/UserFiles/Servers...,0.063418,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
340,346,Hillsboro ISD,Second,docx,https://www.hillsboroisd.org//cms/lib/TX022174...,0.978226,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
376,382,Joaquin ISD,First,pdf,http://www.joaquinisd.net/upload/page/0025/Joa...,0.000045,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
431,439,Liberty Hill ISD,html,html,https://www.libertyhill.txed.net/Domain/294,0.092454,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
480,488,McLeod ISD,Second,pdf,http://www.mcleodisd.net/sites/mcleodisd.net/f...,0.999955,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5


In [134]:
df.reg25_0811.mean()

0.967940813810111

In [135]:
df.iloc[0, 2]
#df[df.district == 'Abbott ISD'].link

'Second'

In [136]:
pd.display.max_colwidth = 100

AttributeError: module 'pandas' has no attribute 'display'